# Song Recommender - SpotiPy

Purpose = extending the 2 dataframes top_100 and not_hot with audio features from spotify
function :
1. input = artist and song title --> return the song id, then create a list to store all the songs id's (limit to 1)
2. input = list of songs-ids and then the function must return the audio features for every song (with a dictionnary)
3. add_audio_featu : pick dataframe -> extend the original dataframe with song - title - audio features

In [194]:
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

In [135]:
not_hot = pd.read_csv('not_hot.csv')
not_hot.shape

(3640, 3)

In [197]:
# Creating a subset of this dataframe to try the functions we create while avoiding being blocked :
# (if it works on the subset, we will apply it on the whole dataframe)
not_hot_subset = not_hot.loc[0:50]
not_hot_subset.shape

(33, 4)

In [196]:
top_100 = pd.read_csv('top_100.csv')
top_100.shape
top_100.head()

,Unnamed: 0,title,artist
0,0,As It Was,Harry Styles
1,1,First Class,Jack Harlow
2,2,Wait For U,Future Featuring Drake & Tems
3,3,Late Night Talking,Harry Styles
4,4,About Damn Time,Lizzo


In [195]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

In [4]:
# Creating search_song() function 

%writefile -a functions.py

def search_song(df) :
    song_ids = []
    for i in range(len(df)) :
        results = sp.search(q="track:"+df['title'][i],limit=1)
        results_size = len(results["tracks"]["items"])
        if results_size == 0:
            song_ids.append(np.nan)
        else:
            if (results["tracks"]["items"][0]['artists'][0]['name'] in df['artist'][i]) or (df['artist'][i] in results["tracks"]["items"][0]['artists'][0]['name']):
                song_id = results["tracks"]["items"][0]["id"]
                song_ids.append(song_id)
            else :
                song_ids.append(np.nan)
    return song_ids

UsageError: Line magic function `%writefile` not found (But cell magic `%%writefile` exists, did you mean that instead?).


In [142]:
song_ids_top_100 = search_song(top_100)
len(song_ids_top_100)

100

In [143]:
# checking if ID's are not repeated :
len(np.unique(np.array(song_ids_top_100)))

96

In [144]:
# apply the function to the subset first
song_ids_not_hot_subset = search_song(not_hot_subset)
len(song_ids_not_hot_subset)

51

In [145]:
len(np.unique(np.array(song_ids_not_hot_subset)))

34

In [164]:
# apply the function to the whole dataframe :
song_ids_not_hot = search_song(not_hot)
len(song_ids_not_hot)

3640

In [146]:
# Then add ID's to my original dataframes :
top_100['id'] = song_ids_top_100
top_100.head()

,Unnamed: 0,title,artist,id
0,0,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7
1,1,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc
2,2,Wait For U,Future Featuring Drake & Tems,59nOXPmaKlBfGMDeOVGrIK
3,3,Late Night Talking,Harry Styles,1qEmFfgcLObUfQm0j1W2CK
4,4,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3


In [148]:
# Same with subset :
not_hot_subset['id'] = song_ids_not_hot_subset
not_hot_subset.head()

/var/folders/tk/75w8q4_s16n1_zv4jdmdlwhh0000gn/T/ipykernel_10923/402743033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_hot_subset['id'] = song_ids_not_hot_subset


,Unnamed: 0,title,artist,id
0,0,Oops!...I Did It Again,Britney Spears,6naxalmIoLFWR0siv8dnQQ
1,1,All The Small Things,blink-182,2m1hi0nfMR9vdGC8UcrnwU
2,2,Breathe,Faith Hill,NaN
3,3,It's My Life,Bon Jovi,0v1XpBHnsbkCn7iJ9Ucr1l
4,4,Bye Bye Bye,*NSYNC,62bOmKYxYg7dhrC6gH9vFn


In [165]:
# And finally with the full dataframe :
not_hot['id'] = song_ids_not_hot
not_hot.head()

,Unnamed: 0,title,artist,id
0,0,Oops!...I Did It Again,Britney Spears,6naxalmIoLFWR0siv8dnQQ
1,1,All The Small Things,blink-182,2m1hi0nfMR9vdGC8UcrnwU
2,2,Breathe,Faith Hill,NaN
3,3,It's My Life,Bon Jovi,0v1XpBHnsbkCn7iJ9Ucr1l
4,4,Bye Bye Bye,*NSYNC,62bOmKYxYg7dhrC6gH9vFn


In [150]:
# Now I need to drop NaN's to be able to continue with audio features :
# for top_100 :
top_100.isna().sum()

Unnamed: 0    0
title         0
artist        0
id            5
dtype: int64

In [151]:
top_100 = top_100[top_100['id'].isna()==False]
top_100.isna().sum()

Unnamed: 0    0
title         0
artist        0
id            0
dtype: int64

In [152]:
# for subset :
not_hot_subset.isna().sum()

Unnamed: 0     0
title          0
artist         0
id            18
dtype: int64

In [153]:
not_hot_subset = not_hot_subset[not_hot_subset['id'].isna()==False]
not_hot_subset.isna().sum()

Unnamed: 0    0
title         0
artist        0
id            0
dtype: int64

In [166]:
# for not_hot :
not_hot.isna().sum()

Unnamed: 0       0
title            0
artist           0
id            1084
dtype: int64

In [167]:
not_hot = not_hot[not_hot['id'].isna()==False]
not_hot.isna().sum()

Unnamed: 0    0
title         0
artist        0
id            0
dtype: int64

In [118]:
# trying with one row:

#mydict = sp.audio_features(song_ids_not_hot_subset)[0]
#mydict
#new_dict = {key : [mydict[key]] for key in list(mydict.keys())}
#for key in new_dict :
    #new_dict[key].append(2)
    
#new_dict
#new_dict['danceability'].append(2)
#new_dict['danceability'][0]

#pd.DataFrame(new_dict)



{'danceability': [0.751, 2],
 'energy': [0.834, 2],
 'key': [1, 2],
 'loudness': [-5.444, 2],
 'mode': [0, 2],
 'speechiness': [0.0437, 2],
 'acousticness': [0.3, 2],
 'instrumentalness': [1.77e-05, 2],
 'liveness': [0.355, 2],
 'valence': [0.894, 2],
 'tempo': [95.053, 2],
 'type': ['audio_features', 2],
 'id': ['6naxalmIoLFWR0siv8dnQQ', 2],
 'uri': ['spotify:track:6naxalmIoLFWR0siv8dnQQ', 2],
 'track_href': ['https://api.spotify.com/v1/tracks/6naxalmIoLFWR0siv8dnQQ', 2],
 'analysis_url': ['https://api.spotify.com/v1/audio-analysis/6naxalmIoLFWR0siv8dnQQ',
  2],
 'duration_ms': [211160, 2],
 'time_signature': [4, 2]}

In [198]:
# Creating the get_audio_features(list_of_songs) function and concatenating the dataframes :

from time import sleep
from random import randint

def get_audio_features(df_input) : 
    list_of_songs = df_input['id']
    empty_dict = {'danceability': [],'energy': [],'key': [],'loudness': [],'mode': [],'speechiness': [],'acousticness': [],
                  'instrumentalness': [],'liveness': [],'valence': [],'tempo': [],'type': [],'id': [],'uri': [],'track_href': [],
                  'analysis_url': [],'duration_ms': [],'time_signature': []}
    for i in range(len(list_of_songs)) :
        audio_features_dict = sp.audio_features(list_of_songs)[i]
        audio_features_final = {key : [audio_features_dict[key]] for key in list(audio_features_dict.keys())}
        for key in empty_dict :
            empty_dict[key].append(audio_features_final[key][0])
        wait_time = randint(3,6)
        sleep(wait_time)
        
    audio_features_df = pd.DataFrame(empty_dict)
    return pd.concat([df_input.reset_index(drop = True), audio_features_df.reset_index(drop = True)], axis=1)

In [160]:
# Let's first try with the top_100 df 

top_100_audio_feat = get_audio_features(top_100)
top_100_audio_feat.tail()

,Unnamed: 0,title,artist,id,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
90,95,Otro Atardecer,Bad Bunny & The Marias,0E0DRHf5PfMeor0ZCwB3oT,0.777,0.595,2,-6.790,1,0.0375,...,0.0669,0.546,108.020,audio_features,0E0DRHf5PfMeor0ZCwB3oT,spotify:track:0E0DRHf5PfMeor0ZCwB3oT,https://api.spotify.com/v1/tracks/0E0DRHf5PfMe...,https://api.spotify.com/v1/audio-analysis/0E0D...,244444,4
91,96,We Cry Together,Kendrick Lamar & Taylour Paige,67XC51nlZncNpHmZ8rOU9a,0.648,0.680,11,-7.276,1,0.3450,...,0.0808,0.504,106.890,audio_features,67XC51nlZncNpHmZ8rOU9a,spotify:track:67XC51nlZncNpHmZ8rOU9a,https://api.spotify.com/v1/tracks/67XC51nlZncN...,https://api.spotify.com/v1/audio-analysis/67XC...,341307,3
92,97,Dos Mil 16,Bad Bunny,4d4ZXH4dr5bYfgErHiZCX2,0.817,0.670,1,-6.534,1,0.0506,...,0.1280,0.495,130.058,audio_features,4d4ZXH4dr5bYfgErHiZCX2,spotify:track:4d4ZXH4dr5bYfgErHiZCX2,https://api.spotify.com/v1/tracks/4d4ZXH4dr5bY...,https://api.spotify.com/v1/audio-analysis/4d4Z...,208615,3
93,98,DFMU,Ella Mai,1bdm32mVmoGcek5bVKxQKd,0.708,0.673,6,-4.163,1,0.0585,...,0.1030,0.311,136.969,audio_features,1bdm32mVmoGcek5bVKxQKd,spotify:track:1bdm32mVmoGcek5bVKxQKd,https://api.spotify.com/v1/tracks/1bdm32mVmoGc...,https://api.spotify.com/v1/audio-analysis/1bdm...,197520,4
94,99,Hate Our Love,Queen Naija & Big Sean,3pEINn37iH5fx8TpXYNGIO,0.662,0.655,2,-5.440,1,0.2600,...,0.1240,0.356,82.079,audio_features,3pEINn37iH5fx8TpXYNGIO,spotify:track:3pEINn37iH5fx8TpXYNGIO,https://api.spotify.com/v1/tracks/3pEINn37iH5f...,https://api.spotify.com/v1/audio-analysis/3pEI...,226374,4


In [161]:
# With the subset :
not_hot_subset_audio_feat = get_audio_features(not_hot_subset)
not_hot_subset_audio_feat.head()

,Unnamed: 0,title,artist,id,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Oops!...I Did It Again,Britney Spears,6naxalmIoLFWR0siv8dnQQ,0.751,0.834,1,-5.444,0,0.0437,...,0.3550,0.894,95.053,audio_features,6naxalmIoLFWR0siv8dnQQ,spotify:track:6naxalmIoLFWR0siv8dnQQ,https://api.spotify.com/v1/tracks/6naxalmIoLFW...,https://api.spotify.com/v1/audio-analysis/6nax...,211160,4
1,1,All The Small Things,blink-182,2m1hi0nfMR9vdGC8UcrnwU,0.434,0.897,0,-4.918,1,0.0488,...,0.6120,0.684,148.726,audio_features,2m1hi0nfMR9vdGC8UcrnwU,spotify:track:2m1hi0nfMR9vdGC8UcrnwU,https://api.spotify.com/v1/tracks/2m1hi0nfMR9v...,https://api.spotify.com/v1/audio-analysis/2m1h...,167067,4
2,3,It's My Life,Bon Jovi,0v1XpBHnsbkCn7iJ9Ucr1l,0.551,0.913,0,-4.063,0,0.0466,...,0.3470,0.544,119.992,audio_features,0v1XpBHnsbkCn7iJ9Ucr1l,spotify:track:0v1XpBHnsbkCn7iJ9Ucr1l,https://api.spotify.com/v1/tracks/0v1XpBHnsbkC...,https://api.spotify.com/v1/audio-analysis/0v1X...,224493,4
3,4,Bye Bye Bye,*NSYNC,62bOmKYxYg7dhrC6gH9vFn,0.610,0.926,8,-4.843,0,0.0479,...,0.0821,0.861,172.638,audio_features,62bOmKYxYg7dhrC6gH9vFn,spotify:track:62bOmKYxYg7dhrC6gH9vFn,https://api.spotify.com/v1/tracks/62bOmKYxYg7d...,https://api.spotify.com/v1/audio-analysis/62bO...,200400,4
4,5,Thong Song,Sisqo,5Mmk2ii6laakqfeCT7OnVD,0.706,0.888,2,-6.959,1,0.0654,...,0.0700,0.714,121.549,audio_features,5Mmk2ii6laakqfeCT7OnVD,spotify:track:5Mmk2ii6laakqfeCT7OnVD,https://api.spotify.com/v1/tracks/5Mmk2ii6laak...,https://api.spotify.com/v1/audio-analysis/5Mmk...,253733,4


In [192]:
# With not_hot df : I need to slice it because it's too long to run, I get an error !
#not_hot.shape # 2556 rows
not_hot1 = not_hot.loc[:1000] # 662 rows
not_hot2 = not_hot.loc[1001:2000] # 776 rows
not_hot3 = not_hot.loc[2001:3001] # 653 rows
not_hot4 = not_hot.loc[3001:3700]

In [200]:
not_hot1_audio_feat = get_audio_features(not_hot1)
not_hot1_audio_feat.head()

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=6naxalmIoLFWR0siv8dnQQ,2m1hi0nfMR9vdGC8UcrnwU,0v1XpBHnsbkCn7iJ9Ucr1l,62bOmKYxYg7dhrC6gH9vFn,5Mmk2ii6laakqfeCT7OnVD,3yfqSUWxFvZELEM4PmlwIR,7H6ev70Weq6DdpZyyTmUXk,49X0LAl6faAusYq02PRAY6,3cqZlRpC4lOnxUE7HE3N5k,2xaFtNhCw5Q4hn1EKsrkLv,79EAVfOpzgfHx3IE2BVFQv,0pMUR7Uvp6vxlbG0qBFvgM,4LwU4Vp6od3Sb08CsP99GC,60a0Rd6pjrkxjPbaKzXjfq,7GHKA8GIMcND6c5nN1sFnD,4Y8q64VnhD0vFYy9g2WFpi,2avKuMN2QXkaG9vvHa2JLt,6Sy9BUbgFse0n0LPA5lwy5,0LWkaEyQRkF0XAms8Bg1fC,4l1MNtoCiTxBwf6yxD7kFd,0AcLrSfAEBQcUnHOTm5pXg,0JcgDliDqDac8iOFFRm7tZ,7m97yKVzPpXV9Z4ezEziOD,5S0cvScXb33JVpjTiY9Ivi,7f1Dmr246cJ9uQYdbplTbh,5jZfGxZDayFA1HzYmyOG3a,7iXF2W9vKmDoGAhlHdpyIa,7mYvtEeBdMqRSyj1Qpv6my,3BsaRV5QIulYz2lV9WWa8T,1rj0XawWJNisX7SGYexowJ,1ak0S3NhwWrUgNlQhJ1412,7JKEA8xYDoFp4q0QBW2PGg,1NNDZc9BHaWe9JqIsPRlNV,1uPrIHgYztXSkkcts9jet8,3w8Mw9GHYepoTWOSdiyosj,0s46SltT4On0Z6kglK1I5c,1RS3LStQYhju50eBdD9Uul,1H5tvpoApNDxvxDexoaAUo,5PMKzsUsTpZZGsCcJBuhP2,0HNXnhNZZtRZFLhP4hfyCP,0Ac0Ge47Up

In [ ]:
not_hot2_audio_feat = get_audio_features(not_hot2)
not_hot2_audio_feat.head()

In [ ]:
not_hot3_audio_feat = get_audio_features(not_hot3)
not_hot3_audio_feat.head()

In [ ]:
not_hot4_audio_feat = get_audio_features(not_hot4)
not_hot4_audio_feat.head()

In [ ]:
not_hot_audio_feat = pd.concat([not_hot1_audio_feat.reset_index(drop = True), not_hot2_audio_feat.reset_index(drop = True), not_hot3_audio_feat.reset_index(drop = True), not_hot4_audio_feat.reset_index(drop = True)], axis=1)
not_hot_audio_feat.shape

In [162]:
# Exporting these dataframes as csv :
top_100_audio_feat.to_csv('top_100_audio_feat.csv')
not_hot_subset_audio_feat.to_csv('not_hot_subset_audio_feat.csv')

In [ ]:
not_hot_audio_feat.to_csv('not_hot_audio_feat.csv')